In [1]:
import os
import logging
import random
import gc
import time
import cv2
import math
import warnings
from pathlib import Path
from datetime import datetime, timezone, timedelta

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import librosa

from sklearn.metrics import roc_auc_score, average_precision_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
import json
import timm

from importlib import reload

logging.basicConfig(level=logging.ERROR)

from module import preprocess_lib, datasets_lib, utils_lib, models_lib, learning_lib, config_lib
reload(config_lib)

<module 'module.config_lib' from '/root/program/birdclef-2025/scripts/module/config_lib.py'>

In [3]:


class CFG:
    def __init__(self, mode="train", kaggle_notebook=False, debug=False):
        assert mode in ["train", "inference"], "mode must be 'train' or 'inference'"
        self.mode = mode
        self.KAGGLE_NOTEBOOK = kaggle_notebook
        self.debug = debug

        # ===== Path Settings =====
        if self.KAGGLE_NOTEBOOK:
            self.OUTPUT_DIR = ''
            self.train_datadir = '/kaggle/input/birdclef-2025/train_audio'
            self.train_csv = '/kaggle/input/birdclef-2025/train.csv'
            self.test_soundscapes = '/kaggle/input/birdclef-2025/test_soundscapes'
            self.submission_csv = '/kaggle/input/birdclef-2025/sample_submission.csv'
            self.taxonomy_csv = '/kaggle/input/birdclef-2025/taxonomy.csv'
            self.spectrogram_npy = '/kaggle/input/birdclef25-mel-spectrograms/birdclef2025_melspec_5sec_256_256.npy'
            self.model_path = '/kaggle/input/birdclef-2025-0330'
        else:
            self.OUTPUT_DIR = '../data/result/'
            self.RAW_DIR = '../data/raw/'
            self.PROCESSED_DIR = '../data/processed/'
            self.train_datadir = '../data/raw/train_audio/'
            self.train_csv = '../data/raw/train.csv'
            self.test_soundscapes = '../data/raw/test_soundscapes/'
            self.submission_csv = '../data/raw/sample_submission.csv'
            self.taxonomy_csv = '../data/raw/taxonomy.csv'
            self.models_dir = "../models/" # 全modelの保存先
            self.model_path = self.models_dir # 各モデルの保存先．学習時に動的に変更．
            
            self.spectrogram_npy = '../data/processed/mel_cleaned_0413/birdclef2025_melspec_5sec_256_256.npy'
            
            self.pseudo_label_csv = "../data/result/pseudo_labels_baseline_7sec.csv"
            self.pseudo_melspec_npy = "../data/processed/train_soundscapes_0407/train_soundscapes_melspecs.npy"

        # ===== Model Settings =====
        self.model_name = 'efficientnet_b0'
        self.pretrained = True if mode == "train" else False
        self.in_channels = 1

        # ===== Audio Settings =====
        self.FS = 32000
        self.WINDOW_SIZE = 5.0 # 推論時のウィンドウサイズ
        self.TARGET_DURATION = 5.0 # データセット作成時のウィンドウサイズ
        self.TARGET_SHAPE = (256, 256)
        self.N_FFT = 1024
        self.HOP_LENGTH = 512
        self.N_MELS = 128
        self.FMIN = 50
        self.FMAX = 14000        

        # ===== Training Mode =====
        if mode == "train":
            self.seed = 42
            self.apex = False
            self.print_freq = 100
            self.num_workers = 2

            self.LOAD_DATA = True
            self.epochs = 10
            self.batch_size = 516
            self.criterion = 'BCEWithLogitsLoss'

            self.n_fold = 5
            self.selected_folds = [0, 1, 2, 3, 4]

            self.optimizer = 'AdamW'
            self.lr = 5e-4
            self.weight_decay = 1e-5
            self.scheduler = 'CosineAnnealingLR'
            self.min_lr = 1e-6
            self.T_max = self.epochs

            self.aug_prob = 0.5
            self.mixup_alpha_real = 0.5
            self.mixup_alpha_pseudo = 0.5
            
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
            
            self.use_pseudo_mixup = False  # pseudo lableでmixupするかどうか
            self.pseudo_mix_prob = 0.4  # mixupでpseudo lableを使う確率
            self.pseudo_conf_threshold = 0.5
            

            if self.debug:
                self.epochs = 2
                self.selected_folds = [0]
                self.batch_size = 4
                

In [4]:
# debug trueにするとvalidationの数が1000に固定される．
cfg = CFG(mode="train", kaggle_notebook=False, debug=False)

In [5]:
utils_lib.set_seed(cfg.seed)

In [10]:
predictions = pd.read_csv("../models/fold0_mel_cleaned0413/predictions_fold0.csv")
targets = pd.read_csv("../models/fold0_mel_cleaned0413/targets_fold0.csv")

In [12]:
row_ids = predictions["row_id"]
# 確率とターゲットをNumPyで取得
preds = predictions.drop(columns=["row_id"]).values
true = targets.drop(columns=["row_id"]).values

# 安定化のためのepsilon
epsilon = 1e-7

# BCE loss をサンプルごとに計算（平均）
bce = - (true * np.log(preds + epsilon) + (1 - true) * np.log(1 - preds + epsilon))
sample_loss = bce.mean(axis=1)  # もしくは .sum(axis=1) でもOK

# DataFrameにまとめる
loss_df = pd.DataFrame({
    "row_id": row_ids,
    "sample_loss": sample_loss
})

# 上位の難しいサンプルを見たい場合：
loss_df = loss_df.sort_values(by="sample_loss", ascending=False).reset_index(drop=True)

loss_df.head(10)  # 上位10件を表示

,row_id,sample_loss
0,blcjay1/XC319947.ogg,0.169229
1,roahaw/XC410261.ogg,0.151585
2,solsan/XC713701.ogg,0.135131
3,grekis/XC317696.ogg,0.123400
4,yebela1/XC288218.ogg,0.121623
5,chfmac1/XC541174.ogg,0.117405
6,roahaw/XC390055.ogg,0.115185
7,baymac/XC1918.ogg,0.111526
8,grasal4/XC562305.ogg,0.109730
9,blcjay1/XC597684.ogg,0.104658


,row_id,sample_loss
0,blcjay1/XC319947.ogg,1.692288e-01
1,roahaw/XC410261.ogg,1.515849e-01
2,solsan/XC713701.ogg,1.351306e-01
3,grekis/XC317696.ogg,1.233998e-01
4,yebela1/XC288218.ogg,1.216227e-01
...,...,...
5708,bobfly1/iNat1274637.ogg,2.511433e-06
5709,wbwwre1/XC311851.ogg,2.335629e-06
5710,bbwduc/iNat152952.ogg,4.937962e-07
5711,bbwduc/iNat938459.ogg,4.356543e-07
